In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib
import warnings
warnings.filterwarnings('ignore')

# --- 1. Load the Dataset ---
try:
    df = pd.read_csv('final_data.csv')
    print("✅ Successfully loaded 'final_data.csv'")
except FileNotFoundError:
    print("❌ 'final_data.csv' not found.")
    exit()

# --- 2. Clean Column Names ---
df.columns = df.columns.str.strip()
print("✅ Column names cleaned.")

# --- 3. Feature Selection & Data Cleaning ---
features = [
    'age', 'height', 'appearance', 'goals', 'assists', 
    'yellow cards', 'red cards', 'goals conceded', 'clean sheets', 
    'minutes played', 'days_injured', 'games_injured'
]
# --- THIS LINE IS NOW FIXED ---
target = 'current_value'

# Create a clean dataframe for training, ensuring all selected columns exist
df_model = df[features + [target]].copy()
df_model.dropna(inplace=True)
df_model = df_model[df_model['current_value'] > 0] # Use the correct column name here too

print(f"Prepared {len(df_model)} clean data points for training.")

# --- 4. Split Data for Training and Testing ---
X = df_model[features]
y = df_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Data split into {len(X_train)} training samples and {len(X_test)} testing samples.")

# --- 5. Train the XGBoost Model ---
print("\n💪 Training the XGBoost model...")
xgbr = xgb.XGBRegressor(
    objective='reg:squarederror', n_estimators=1000, learning_rate=0.05,
    early_stopping_rounds=10, eval_metric='rmse', n_jobs=-1
)
xgbr.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

# --- 6. Evaluate the Model's Performance ---
predictions = xgbr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"✅ Model training complete!")
print(f"The model's average prediction error (RMSE) is: € {rmse:,.0f}")

# --- 7. Save the Trained Model ---
joblib.dump(xgbr, 'valuation_model.joblib')
print("\n✅ Final model saved to 'valuation_model.joblib'")

✅ Successfully loaded 'final_data.csv'
✅ Column names cleaned.
Prepared 10587 clean data points for training.
Data split into 8469 training samples and 2118 testing samples.

💪 Training the XGBoost model...
✅ Model training complete!
The model's average prediction error (RMSE) is: € 7,396,774

✅ Final model saved to 'valuation_model.joblib'
